In [77]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

'''
df_full_ipo_date_97_to_19_silicon_valley - This is the data that I scraped from the NASDAQ website that lists all the
IPO's in Silicon Valley betnween 1997 and 2019/04. I will need to add updated IPO results if any new IPO's are announced
but for right now this is static --> REAL TIME WAY TO UPDATE WHEN NEW IPOS ARE ANNOUNCED
zip_all_census_data_silicon_valley - census data from all cities and beaureaus -- that holds econometric stats but also 
race, sex, and age stats about each zipcode
df_all_home_by_zip_silicon_valley - All home prices from 1996 to 2019
'''


"\ndf_full_ipo_date_97_to_19_silicon_valley - This is the data that I scraped from the NASDAQ website that lists all the\nIPO's in Silicon Valley betnween 1997 and 2019/04. I will need to add updated IPO results if any new IPO's are announced\nbut for right now this is static --> REAL TIME WAY TO UPDATE WHEN NEW IPOS ARE ANNOUNCED\nzip_all_census_data_silicon_valley - census data from all cities and beaureaus -- that holds econometric stats but also \nrace, sex, and age stats about each zipcode\ndf_all_home_by_zip_silicon_valley - All home prices from 1996 to 2019\n"

In [78]:
'''
data processing to only view california real estate data

df_median_all_home_price_by_zip = pd.read_csv('data/raw/Zip_MedianListingPrice_AllHomes.csv',  encoding="ISO-8859-1")
df_single_family_residence_by_zip = pd.read_csv('data/raw/Zip_Zhvi_SingleFamilyResidence.csv',  encoding="ISO-8859-1")

df_median_all_home_price_by_zip_california = df_median_all_home_price_by_zip[df_median_all_home_price_by_zip['State']=='CA']
df_single_family_residence_by_zip_california = df_single_family_residence_by_zip.loc[df_single_family_residence_by_zip['State']=='CA']
'''
#df_sale_counts_by_zip = pd.read_csv('data/raw/Sale_Counts_Zip.csv', encoding="ISO-8859-1")
df_all_home_by_zip = pd.read_csv('data/raw/Zip_Zhvi_AllHomes.csv',  encoding="ISO-8859-1")

#df_sale_counts_by_zip_california = df_sale_counts_by_zip.loc[df_sale_counts_by_zip['StateName']=='California']
df_all_home_by_zip_california = df_all_home_by_zip.loc[df_all_home_by_zip['State']=='CA']


In [79]:
# data processing to load all IPO Data between 1997 and present data. This data has been scraped using code from src/web_scrapers.py
df_full_ipo_date_97_to_19 = pd.read_csv('data/processed/1997-04_2019_full_ipo_data.csv',encoding="ISO-8859-1")

df_ritter_ipo_data_97_to_19 = pd.read_csv('data/raw/ipo_ritter_data.csv',encoding="ISO-8859-1")


In [80]:
#data processing of census data to create demographics by zip code
zip_census_data = pd.read_csv('data/raw/zip_census_bureau_economic_characteristics_2017.csv',encoding="ISO-8859-1")
zip_census_age_race_data = pd.read_csv('data/raw/zip_census_bureau_age_race_2017.csv',encoding="ISO-8859-1")

In [81]:
# type wrangling of all pandas series in each dataframe
#df_sale_counts_by_zip_california["RegionID"] = df_sale_counts_by_zip_california["RegionID"].astype(dtype=np.int64)
#df_sale_counts_by_zip_california['StateName'] = df_sale_counts_by_zip_california['StateName'].astype('str')

df_full_ipo_date_97_to_19['Date Filed'] =  pd.to_datetime(df_full_ipo_date_97_to_19['Date Filed'], errors='coerce', format='%m/%d/%Y')
df_full_ipo_date_97_to_19['Lockup Expiration Date'] =  pd.to_datetime(df_full_ipo_date_97_to_19['Lockup Expiration Date'], errors='coerce', format='%m/%d/%Y')



In [82]:
# data wrangling of census data

zip_census_data.rename(columns = {'Id2':'Zipcode'}, inplace=True)
zip_census_data.rename(columns = {'Percent; EMPLOYMENT STATUS - Civilian labor force - Unemployment Rate':'Unemployment Rate'}, inplace=True)
zip_census_data.rename(columns = {'Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - Less than $10,000':'l10000'}, inplace=True)
zip_census_data.rename(columns = {'Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $10,000 to $14,999':'l15000'}, inplace=True)
zip_census_data.rename(columns = {'Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $15,000 to $24,999':'l25000'}, inplace=True)
zip_census_data.rename(columns = {'Estimate; COMMUTING TO WORK - Mean travel time to work (minutes)':'Mean Travel Time to Work Estimate (minutes)'}, inplace=True)
zip_census_data.rename(columns = {'Percent; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - $200,000 or more':'Percent of Households with Income Greater than $200,000'}, inplace=True)
zip_census_data.rename(columns = {'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - Median household income (dollars)':'Median Household Income Estimate (dollars)'}, inplace=True)
zip_census_data.rename(columns = {'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Total households - Mean household income (dollars)':'Mean Household Income Estimate (dollars)'}, inplace=True)
zip_census_data.rename(columns = {'Estimate; INCOME AND BENEFITS (IN 2017 INFLATION-ADJUSTED DOLLARS) - Per capita income (dollars)':'Per Capita Income Estimate (dollars)'}, inplace=True)
zip_census_data.rename(columns = {'Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population - No health insurance coverage':'Percent of Population with no Health Insurance Coverage'}, inplace=True)
zip_census_data.rename(columns = {'Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All people':'Percent of People whose Income in the Past 12 months has been Below Poverty Level'}, inplace=True)


filtered_census_data_columns = ['Zipcode',
                               'Unemployment Rate',
                               'Mean Travel Time to Work Estimate (minutes)',
                               'Percent of Households with Income Greater than $200,000',
                               'Median Household Income Estimate (dollars)',
                               'Mean Household Income Estimate (dollars)',
                               'Per Capita Income Estimate (dollars)',
                               'Percent of Population with no Health Insurance Coverage',
                               'Percent of People whose Income in the Past 12 months has been Below Poverty Level',
                               'Percent of Households With Income Less Than $24,999']
zip_census_data['l10000'].replace("-","0.0", regex=True, inplace=True)
zip_census_data['l10000'].replace("N","0.0", regex=True, inplace=True)
zip_census_data['l10000'] = zip_census_data['l10000'].astype(float)

zip_census_data['l15000'].replace("-","0.0", regex=True, inplace=True)
zip_census_data['l15000'].replace("N","0.0", regex=True, inplace=True)
zip_census_data['l15000'] = zip_census_data['l15000'].astype(float)

zip_census_data['l25000'].replace("-","0.0", regex=True, inplace=True)
zip_census_data['l25000'].replace("N","0.0", regex=True, inplace=True)
zip_census_data['l25000'] = zip_census_data['l25000'].astype(float)
zip_census_data["Percent of Households With Income Less Than $24,999"] = zip_census_data['l10000'] + zip_census_data['l15000']+zip_census_data['l25000']
zip_census_data = zip_census_data.replace(['250,000+'], ['250000'], regex=True)

zip_census_data = zip_census_data.filter(filtered_census_data_columns)

zip_census_age_race_data.rename(columns = {'Id2':'Zipcode'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Estimate; SEX AND AGE - Median age (years)':'Median Age'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; SEX AND AGE - Under 18 years':'Percent of People under 18 years of age'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; SEX AND AGE - 65 years and over':'Percent of People 65 years and over'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; SEX AND AGE - 18 years and over - Male':'Percent of Males'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; SEX AND AGE - 18 years and over - Female':'Percent of Females'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; HISPANIC OR LATINO AND RACE - Total population - Hispanic or Latino (of any race)':'Percent of People who are Hispanic'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - White alone':'Percent of People who are White'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Black or African American alone':'Percent of People who are Black or African American'}, inplace=True)
zip_census_age_race_data.rename(columns = {'Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Asian alone':'Percent of People who are Asian'}, inplace=True)

filtered_census_age_race_data_columns = ['Zipcode',
                               'Median Age',
                               'Percent of People under 18 years of age',
                               'Percent of People 65 years and over',
                               'Percent of Males',
                               'Percent of Females',
                               'Percent of People who are Hispanic',
                               'Percent of People who are White',
                               'Percent of People who are Black or African American',
                               'Percent of People who are Asian']
zip_census_age_race_data = zip_census_age_race_data.filter(filtered_census_age_race_data_columns)


In [7]:
'''
***DONT RUN THE NEXT 3 IF THESE ARE ALREADY CREATED***
creating list and dictionary for storing all the zipcodes in silicon valley + san Francisco
'''
import csv
zipcodes = {} # structured with within 5, 10 miles from another zipcode
zip_list=[]

def add_zipcodes(filename):
    with open(filename, 'r', encoding='utf-8-sig') as f:
        reader = csv.reader(f)
        your_list = list(reader)
        for z in range(0, len(your_list)):
            zipcodes[your_list[z][0]]= [],[]
            zip_list.append(your_list[z][0])

with open('data/raw/Santa_Clara_County_Zipcodes.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    your_list = list(reader)
for z in range(0, len(your_list)):
    zipcodes[your_list[z][0]]= [],[]
    zip_list.append(your_list[z][0])
    
with open('data/raw/San_Mateo_County_Zipcodes.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    your_list = list(reader)
for z in range(0, len(your_list)):
    zipcodes[your_list[z][0]]= [],[]
    zip_list.append(your_list[z][0])
    
with open('data/raw/San_Francisco_County_Zipcodes.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    your_list = list(reader)
for z in range(0, len(your_list)):
    zipcodes[your_list[z][0]]= [],[]
    zip_list.append(your_list[z][0])
    
with open('data/raw/Alameda_County_Zipcodes.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    your_list = list(reader)
for z in range(0, len(your_list)):
    zipcodes[your_list[z][0]]= [],[]
    zip_list.append(your_list[z][0])



len(zip_list)

173

In [8]:
import urllib.request
import json

def calculate_distance_between_zips(zipcode,min_radius, max_radius):
    # api-endpoint 
    URL_base = "https://api.zip-codes.com/ZipCodesAPI.svc/1.0/FindZipCodesInRadius?zipcode="
  
    URL = URL_base+zipcode+'&minimumradius='+min_radius+'&maximumradius='+max_radius+'&key=DEMOAPIKEY'
    # sending get request and saving the response as response object 
    contents = urllib.request.urlopen(URL).read()
  

    # printing the output 
    zipcodes_nearby =[]
    for i in range(1, len(json.loads(contents)['DataList'])):
        zipcodes_nearby.append(json.loads(contents)['DataList'][i]['Code'])
        
    return zipcodes_nearby

In [9]:
'''
***DONT RUN IF THESE ARE ALREADY CREATED***
Create zipcodes distance list
'''
for i in range(0, len(zip_list)):
    print(zip_list[i])
    zipcodes[zip_list[i]]= calculate_distance_between_zips(zip_list[i], '0','5'), calculate_distance_between_zips(zip_list[i], '5','10')
    

95002
95008
95013
95014
95020
95023
94550
94022
94024
95030
95032
95033
95035
95037
95140
94040
94041
94043
94301
94303
94304
94306
94028
95110
95111
95112
95113
95116
95117
95118
95119
95120
95121
95122
95123
95124
95125
95126
95127
95128
95129
95130
95131
95132
95133
95134
95135
95136
95138
95139
95148
95046
95050
95051
95054
95053
95070
94305
94085
94086
94087
94089
95076
94027
94002
94005
94010
94014
94015
94019
94020
94021
94025
94030
94037
94038
94044
94303
94060
94028
94061
94062
94063
94065
94066
94070
94128
94074
94401
94402
94403
94404
94080
94102
94103
94104
94105
94107
94108
94109
94110
94111
94112
94114
94115
94116
94117
94118
94121
94122
94123
94124
94127
94129
94130
94131
94132
94133
94134
94158
94501
94502
94706
94702
94703
94704
94705
94707
94708
94709
94710
94720
94514
94546
94552
94505
94568
94608
94536
94538
94539
94555
94541
94542
94544
94545
94550
94551
94560
94601
94602
94603
94605
94606
94607
94609
94610
94611
94612
94618
94619
94621
94613
94566
94588
94577
9457

In [83]:
from datetime import datetime  
from datetime import timedelta  

'''
processing to merge all zip code data into one dataframe to create a feature
from 
1) df_all_home_by_zip_silicon_valley
2) zip_all_census_data_silicon_valley
'''
 
zip_census_age_race_data_silicon_valley = zip_census_age_race_data[zip_census_age_race_data['Zipcode'].isin(zip_list)]
zip_census_data_silicon_valley = zip_census_data[zip_census_data['Zipcode'].isin(zip_list)]
#df_sale_counts_by_zip_silicon_valley = df_sale_counts_by_zip_california[df_sale_counts_by_zip_california['RegionName'].isin(zip_list)]
# the one we should use
df_all_home_by_zip_silicon_valley = df_all_home_by_zip_california[df_all_home_by_zip_california['RegionName'].isin(zip_list)]
# df_single_family_residence_by_zip_silicon_valley only includes detached homes, so super expensive
#df_single_family_residence_by_zip_silicon_valley = df_single_family_residence_by_zip_california[df_single_family_residence_by_zip_california['RegionName'].isin(zip_list)]
#df_median_all_home_price_by_zip_silicon_valley = df_median_all_home_price_by_zip_california[df_median_all_home_price_by_zip_california['RegionName'].isin(zip_list)]
df_full_ipo_date_97_to_19_silicon_valley = df_full_ipo_date_97_to_19[df_full_ipo_date_97_to_19['Zipcode'].isin(zip_list)]

df_full_ipo_date_97_to_19_silicon_valley = df_full_ipo_date_97_to_19_silicon_valley.drop(['Lockup Expiration Date', 'Lockup Period'], axis=1)

df_all_home_by_zip_silicon_valley = df_all_home_by_zip_silicon_valley.drop(['City', 'RegionID','State','Metro','CountyName', 'SizeRank'], axis=1)
#df_sale_counts_by_zip_silicon_valley = df_sale_counts_by_zip_silicon_valley.drop(['RegionID','StateName','SizeRank', 'seasAdj'], axis=1)

# causing warning -- issues with grabbing the 180, but all lockup periods here are 180
df_full_ipo_date_97_to_19_silicon_valley['Lockup Expiration Date'] = df_full_ipo_date_97_to_19_silicon_valley['Date Filed']+ timedelta(days=180)


In [96]:
export_csv = df_zip_all_joined_data.to_csv ('data/processed/df_zip_all_joined_data.csv', index = True, header=True) #Don't forget to add '.csv' at the end of the path
export_csv = df_all_home_by_zip_silicon_valley.to_csv ('data/processed/df_all_home_by_zip_silicon_valley.csv', index = True, header=True) #Don't forget to add '.csv' at the end of the path
export_csv = zip_census_data_silicon_valley.to_csv ('data/processed/export_dataframe.csv', index = True, header=True) #Don't forget to add '.csv' at the end of the path
export_csv = df_full_ipo_date_97_to_19_silicon_valley.to_csv ('data/processed/df_full_ipo_date_97_to_19_silicon_valley.csv', index = True, header=True) #Don't forget to add '.csv' at the end of the path
export_csv = ipo_final_df.to_csv ('data/processed/ipo_final_df.csv', index = True, header=True) #Don't forget to add '.csv' at the end of the path


with open('data/processed/zipcodes_within_radius.txt', 'w') as json_file:
  json.dump(zipcodes, json_file)


In [84]:
'''
*** DONT RUN MULTIPLE TIMES. ALL HOMES AND SALES COUNTS PREFIX WILL BE ADDED MULTIPLE TIMES***
1) join census bureau dataframes on -- only joining now to minimize data to silicon valley zipcodes that we will use.
key is zip code
2) join with df_all_home_by_zip_silicon_valley
3) join with df_sale_counts_by_zip_silicon_valley
'''
df_all_home_by_zip_silicon_valley.columns = df_all_home_by_zip_silicon_valley.columns.str.replace('All Homes ', '')
#df_sale_counts_by_zip_silicon_valley.columns = df_sale_counts_by_zip_silicon_valley.columns.str.replace('Sales Counts ', '')

df_all_home_by_zip_silicon_valley = df_all_home_by_zip_silicon_valley.add_prefix('All Homes ')
#df_sale_counts_by_zip_silicon_valley = df_sale_counts_by_zip_silicon_valley.add_prefix('Sales Counts ')
zip_all_census_data_silicon_valley = zip_census_age_race_data_silicon_valley.set_index('Zipcode').join(zip_census_data_silicon_valley.set_index('Zipcode'))

df_all_home_by_zip_silicon_valley.rename(columns = {'All Homes RegionName':'Zipcode'}, inplace=True)
#df_sale_counts_by_zip_silicon_valley.rename(columns = {'Sales Counts RegionName':'Zipcode'}, inplace=True)




In [85]:
#df_zip_all_joined_data = zip_all_census_data_silicon_valley.set_index('Zipcode').join(df_all_home_by_zip_silicon_valley.set_index('Zipcode'))
#df_zip_all_joined_data = df_zip_all_joined_data.set_index('Zipcode').join(df_sale_counts_by_zip_silicon_valley.set_index('Zipcode'))

df_zip_all_joined_data = pd.merge(zip_all_census_data_silicon_valley, df_all_home_by_zip_silicon_valley, on='Zipcode', how='inner')
#df_zip_all_joined_data = pd.merge(df_zip_all_joined_data, df_sale_counts_by_zip_silicon_valley, on='Zipcode', how='inner')


In [86]:
'''
wrangling ritter data
'''
df_ritter_ipo_data_97_to_19.rename(columns = {'Ticker':'Symbol'}, inplace=True)

df_ritter_ipo_data_97_to_19["Found"] = df_ritter_ipo_data_97_to_19["Found"].astype(dtype=np.int64)





In [87]:
df_full_ipo_date_97_to_19_silicon_valley = df_full_ipo_date_97_to_19_silicon_valley.set_index('Symbol').join(df_ritter_ipo_data_97_to_19.set_index('Symbol'), how='left')


In [88]:
df_full_ipo_date_97_to_19_silicon_valley = df_full_ipo_date_97_to_19_silicon_valley.drop(['IPO Name', 'Offer date','CUSIP','PERM'], axis=1)

df_full_ipo_date_97_to_19_silicon_valley.drop_duplicates(subset='CIK', keep='first', inplace=True)




In [89]:
df_zip_all_joined_data = df_zip_all_joined_data.replace(['\,'], [''], regex=True)
df_zip_all_joined_data = df_zip_all_joined_data.replace(['\+'], [''], regex=True)


In [90]:
zip_df_columns_list =  list(df_zip_all_joined_data.columns.values)
#zip_df_columns_list.remove('Zipcode')

In [91]:

if 'Zipcode' in filtered_census_data_columns: 
    filtered_census_data_columns.remove('Zipcode')
    
if 'Zipcode' in filtered_census_age_race_data_columns: 
    filtered_census_age_race_data_columns.remove('Zipcode')
ipo_header_list = list(df_full_ipo_date_97_to_19_silicon_valley.columns.values) + filtered_census_age_race_data_columns+ filtered_census_data_columns+  ['All Homes Date Filed', 
                                                                                      'All Homes Lockup Expiration Date',
                                                                                      'All Homes 1 Year After Date Filed', 
                                                                                      'All Homes 2 Years After Date Filed']





In [92]:
def join_IPO_and_Zip_Data(IPO_Date_Filed, IPO_Lockup_Expiration_Date):
    filtered_columns = filtered_census_age_race_data_columns+ filtered_census_data_columns #remove 'zipcode'
    ipo_month_filed = IPO_Date_Filed.month
    ipo_year_filed = IPO_Date_Filed.year
    AllHomes_header_filed = 'All Homes ' +str(ipo_year_filed)+'-'+str(ipo_month_filed).zfill(2)
    #SalesCounts_header_filed = 'Sales Counts ' +str(ipo_year_filed)+'-'+str(ipo_month_filed).zfill(2)
    
    ipo_month = IPO_Lockup_Expiration_Date.month
    ipo_year = IPO_Lockup_Expiration_Date.year
    AllHomes_header_lockup = 'All Homes ' +str(ipo_year)+'-'+str(ipo_month).zfill(2)
    #SalesCounts_header_lockup = 'Sales Counts ' +str(ipo_year)+'-'+str(ipo_month).zfill(2)
    
    AllHomes_header_filed_1_yr = 'All Homes ' +str(int(ipo_year_filed)+1)+'-'+str(ipo_month_filed).zfill(2)
   # SalesCounts_header_filed_1_yr = 'Sales Counts ' +str(int(ipo_year_filed)+1)+'-'+str(ipo_month_filed).zfill(2)
    
    AllHomes_header_filed_2_yr = 'All Homes ' +str(int(ipo_year_filed)+2)+'-'+str(ipo_month_filed).zfill(2)
    #SalesCounts_header_filed_2_yr = 'Sales Counts ' +str(int(ipo_year_filed)+2)+'-'+str(ipo_month_filed).zfill(2)
    
    filtered_columns = filtered_columns + [AllHomes_header_filed,
                                           AllHomes_header_lockup,
                                            AllHomes_header_filed_1_yr, 
                                            AllHomes_header_filed_2_yr]
    return filtered_columns

In [93]:
'''
Distance from IPO   = estimate is .2 if in the same zipcode as IPO
                    = estimate is 0.5 if not in same zip code as IPO and less than 5 miles from zipcode to IPO
                    = estimate is 1 if greater than 5 and less than 10 miles from zipcode to IPO
zipcodes['94555']
'''
new_df_list =[]

count = 0 
for index, row in df_full_ipo_date_97_to_19_silicon_valley.iterrows():
    ipo_zipcode = row['Zipcode']
    zipcode_row = df_zip_all_joined_data.loc[df_zip_all_joined_data['Zipcode']== int(ipo_zipcode)]
    headerList = join_IPO_and_Zip_Data(row['Date Filed'], row['Lockup Expiration Date'])
    data = np.concatenate((np.array(row.values), zipcode_row.filter(headerList).values), axis=None)
    dictionary = dict(zip(ipo_header_list, data))
    dictionary['Symbol'] = index
    dictionary['Distance to IPO'] = .2
    dictionary['Zipcode for Distance'] = ipo_zipcode
    new_df_list.append(dictionary)
    within_5miles = zipcodes[ipo_zipcode][0]
    within_10miles = zipcodes[ipo_zipcode][1]
    for i in range(0, len(within_5miles)):
        zipcode_row = df_zip_all_joined_data.loc[df_zip_all_joined_data['Zipcode']== int(within_5miles[i])]
        data = np.concatenate((np.array(row.values), zipcode_row.filter(headerList).values), axis=None)
        dictionary = dict(zip(ipo_header_list, data))
        dictionary['Symbol'] = index
        dictionary['Distance to IPO'] = .5
        dictionary['Zipcode for Distance'] = within_5miles[i]
        new_df_list.append(dictionary)

    for j in range(0, len(within_10miles)):
        zipcode_row = df_zip_all_joined_data.loc[df_zip_all_joined_data['Zipcode']== int(within_10miles[j])]
        data = np.concatenate((np.array(row.values), zipcode_row.filter(headerList).values), axis=None)
        dictionary = dict(zip(ipo_header_list, data))
        dictionary['Symbol'] = index
        dictionary['Distance to IPO'] = 1
        dictionary['Zipcode for Distance'] = within_10miles[j]
        new_df_list.append(dictionary)

    
ipo_final_df = pd.DataFrame(new_df_list)

In [94]:

ipo_final_df.dropna(subset=['Median Age'], how='all', inplace = True)

ipo_final_df = ipo_final_df.replace(['--'], [''], regex=True)
ipo_final_df= ipo_final_df.replace(r'^\s*$', np.nan, regex=True)
ipo_final_df = ipo_final_df.replace(['\,'], [''], regex=True)
ipo_final_df = ipo_final_df.replace(['\+'], [''], regex=True)


In [95]:
normalization_list = ['Offer Amount', 'Number of Employees','Found', 'Median Age',
 'Percent of People under 18 years of age',
 'Percent of People 65 years and over',
 'Percent of Males',
 'Percent of Females',
 'Percent of People who are Hispanic',
 'Percent of People who are White',
 'Percent of People who are Black or African American',
 'Percent of People who are Asian',
 'Unemployment Rate',
'Mean Travel Time to Work Estimate (minutes)',
'Percent of Households with Income Greater than $200,000',
'Median Household Income Estimate (dollars)',
'Mean Household Income Estimate (dollars)',
'Per Capita Income Estimate (dollars)',
'Percent of Population with no Health Insurance Coverage',
'Percent of People whose Income in the Past 12 months has been Below Poverty Level',
'Percent of Households With Income Less Than $24,999']




In [74]:
scaler = MinMaxScaler()
ipo_final_df[normalization_list] = scaler.fit_transform(ipo_final_df[normalization_list])

In [75]:
ipo_final_df

,All Homes 1 Year After Date Filed,All Homes 2 Years After Date Filed,All Homes Date Filed,All Homes Lockup Expiration Date,CIK,Company Name,Date Filed,Distance to IPO,Found,Lockup Expiration Date,...,Percent of People who are Asian,Percent of People who are Black or African American,Percent of People who are Hispanic,Percent of People who are White,Percent of People whose Income in the Past 12 months has been Below Poverty Level,Percent of Population with no Health Insurance Coverage,Symbol,Unemployment Rate,Zipcode,Zipcode for Distance
0,481900.0,487400.0,359400.0,420100.0,1090872,AGILENT TECHNOLOGIES INC,1999-08-16,0.2,0.585366,2000-02-12,...,0.634014,0.066210,0.191718,0.308051,0.058468,0.184358,A,0.416667,95051,95051
4,467300.0,461900.0,348200.0,407300.0,1090872,AGILENT TECHNOLOGIES INC,1999-08-16,0.5,0.585366,2000-02-12,...,0.331973,0.082192,0.320552,0.462077,0.264113,0.379888,A,0.351852,95051,95050
5,698300.0,671100.0,520400.0,608400.0,1090872,AGILENT TECHNOLOGIES INC,1999-08-16,0.5,0.585366,2000-02-12,...,0.650340,0.029680,0.064417,0.415403,0.060484,0.162011,A,0.361111,95051,94087
6,562500.0,563000.0,417500.0,488600.0,1090872,AGILENT TECHNOLOGIES INC,1999-08-16,0.5,0.585366,2000-02-12,...,0.585034,0.034247,0.225460,0.344224,0.098790,0.229050,A,0.574074,95051,94086
8,484800.0,498900.0,389000.0,437100.0,1090872,AGILENT TECHNOLOGIES INC,1999-08-16,0.5,0.585366,2000-02-12,...,0.321088,0.132420,0.384969,0.397900,0.219758,0.446927,A,0.638889,95051,95117
10,653400.0,651800.0,476600.0,558700.0,1090872,AGILENT TECHNOLOGIES INC,1999-08-16,0.5,0.585366,2000-02-12,...,0.872109,0.031963,0.049080,0.257876,0.064516,0.206704,A,0.425926,95051,95129
12,404900.0,429100.0,316700.0,360200.0,1090872,AGILENT TECHNOLOGIES INC,1999-08-16,0.5,0.585366,2000-02-12,...,0.508844,0.043379,0.475460,0.232205,0.133065,0.441341,A,0.388889,95051,94085
13,464700.0,490000.0,356200.0,411200.0,1090872,AGILENT TECHNOLOGIES INC,1999-08-16,0.5,0.585366,2000-02-12,...,0.765986,0.095890,0.115031,0.249708,0.036290,0.206704,A,0.351852,95051,95054
14,444800.0,447900.0,341900.0,395400.0,1090872,AGILENT TECHNOLOGIES INC,1999-08-16,0.5,0.585366,2000-02-12,...,0.194558,0.095890,0.535276,0.408401,0.173387,0.502793,A,0.416667,95051,95128
15,427300.0,431900.0,325900.0,378200.0,1090872,AGILENT TECHNOLOGIES INC,1999-08-16,0.5,0.585366,2000-02-12,...,0.209524,0.118721,0.490798,0.413069,0.217742,0.446927,A,0.546296,95051,95126


In [28]:
ipo_header_list

['Company Name',
 'Offer Amount',
 'Date Filed',
 'Zipcode',
 'Number of Employees',
 'CIK',
 'Lockup Expiration Date',
 'Found',
 'Unemployment Rate',
 'Mean Travel Time to Work Estimate (minutes)',
 'Percent of Households with Income Greater than $200,000',
 'Median Household Income Estimate (dollars)',
 'Mean Household Income Estimate (dollars)',
 'Per Capita Income Estimate (dollars)',
 'Percent of Population with no Health Insurance Coverage',
 'Percent of People whose Income in the Past 12 months has been Below Poverty Level',
 'Percent of Households With Income Less Than $24,999',
 'Median Age',
 'Percent of People under 18 years of age',
 'Percent of People 65 years and over',
 'Percent of Males',
 'Percent of Females',
 'Percent of People who are Hispanic',
 'Percent of People who are White',
 'Percent of People who are Black or African American',
 'Percent of People who are Asian',
 'All Homes Date Filed',
 'All Homes Lockup Expiration Date',
 'All Homes 1 Year After Date Fil

In [59]:
zip_census_data

,Zipcode,Unemployment Rate,Mean Travel Time to Work Estimate (minutes),"Percent of Households with Income Greater than $200,000",Median Household Income Estimate (dollars),Mean Household Income Estimate (dollars),Per Capita Income Estimate (dollars),Percent of Population with no Health Insurance Coverage,Percent of People whose Income in the Past 12 months has been Below Poverty Level,"Percent of Households With Income Less Than $24,999"
0,89010,1.4,10.3,0,46111,56505,25603,28.9,7.8,20.6
1,89019,12.5,28.1,1.6,35856,55862,26622,6.7,16.5,32.3
2,89046,1.9,11.9,4.7,31389,48033,29959,20.9,9.3,24.5
3,89060,13.3,28.2,0.4,38370,47664,20789,16.4,22.5,30.4
4,89061,2.2,31.3,0.9,54375,65354,29760,4.5,5.6,17.9
5,89439,10.5,21.4,7.5,56917,82409,37676,15.4,8.1,10.1
6,90001,11.2,32.4,0.4,35660,45650,11882,20.7,31.9,34.6
7,90002,12.3,34.9,0.4,34000,44463,11807,20.9,33.8,38.4
8,90003,12.5,35.7,0.3,34397,43342,11305,21.8,33.1,38.0
9,90004,7.1,32.1,7.6,46581,82091,30590,24.1,19.2,24.6
